In [1]:
import pandas as pd

In [29]:
aa = pd.read_csv('./Stock Data/Data/stock_df/stock_df/CJ제일제당_stock.csv')

In [32]:
a1 = aa[['Date','Open','High','Low','Close','Volume']]
a1.columns = ['date','open','high','low','close','volume']
a1

,date,open,high,low,close,volume
0,2014-01-02,277500,281000,276000,279500,101630
1,2014-01-03,278000,280000,274500,278500,78910
2,2014-01-06,277000,278000,272500,277000,42426
3,2014-01-07,274000,277000,273000,275500,49898
4,2014-01-08,274500,276500,272000,274500,42155
...,...,...,...,...,...,...
1466,2019-12-23,244000,244500,240000,243500,33748
1467,2019-12-24,245500,245500,241500,242000,17060
1468,2019-12-26,244000,245000,241500,243000,32651
1469,2019-12-27,246000,251500,241000,250000,50065


In [33]:
import numpy as np


COLUMNS_CHART_DATA = ['date', 'open', 'high', 'low', 'close', 'volume']

COLUMNS_TRAINING_DATA_V1 = [
    'open_lastclose_ratio', 'high_close_ratio', 'low_close_ratio',
    'close_lastclose_ratio', 'volume_lastvolume_ratio',
    'close_ma5_ratio', 'volume_ma5_ratio',
    'close_ma10_ratio', 'volume_ma10_ratio',
    'close_ma20_ratio', 'volume_ma20_ratio',
    'close_ma60_ratio', 'volume_ma60_ratio',
    'close_ma120_ratio', 'volume_ma120_ratio',
]

COLUMNS_TRAINING_DATA_V1_RICH = [
    'open_lastclose_ratio', 'high_close_ratio', 'low_close_ratio',
    'close_lastclose_ratio', 'volume_lastvolume_ratio',
    'close_ma5_ratio', 'volume_ma5_ratio',
    'close_ma10_ratio', 'volume_ma10_ratio',
    'close_ma20_ratio', 'volume_ma20_ratio',
    'close_ma60_ratio', 'volume_ma60_ratio',
    'close_ma120_ratio', 'volume_ma120_ratio',
    'inst_lastinst_ratio', 'frgn_lastfrgn_ratio',
    'inst_ma5_ratio', 'frgn_ma5_ratio',
    'inst_ma10_ratio', 'frgn_ma10_ratio',
    'inst_ma20_ratio', 'frgn_ma20_ratio',
    'inst_ma60_ratio', 'frgn_ma60_ratio',
    'inst_ma120_ratio', 'frgn_ma120_ratio',
]

COLUMNS_TRAINING_DATA_V2 = [
    'per', 'pbr', 'roe',
    'open_lastclose_ratio', 'high_close_ratio', 'low_close_ratio',
    'close_lastclose_ratio', 'volume_lastvolume_ratio',
    'close_ma5_ratio', 'volume_ma5_ratio',
    'close_ma10_ratio', 'volume_ma10_ratio',
    'close_ma20_ratio', 'volume_ma20_ratio',
    'close_ma60_ratio', 'volume_ma60_ratio',
    'close_ma120_ratio', 'volume_ma120_ratio',
    'market_kospi_ma5_ratio', 'market_kospi_ma20_ratio', 
    'market_kospi_ma60_ratio', 'market_kospi_ma120_ratio', 
    'bond_k3y_ma5_ratio', 'bond_k3y_ma20_ratio', 
    'bond_k3y_ma60_ratio', 'bond_k3y_ma120_ratio'
]

In [52]:
def preprocess(data, ver='v2'):
    windows = [5, 10, 20, 60, 120]
    for window in windows:
        data['close_ma{}'.format(window)] = \
            data['close'].rolling(window).mean()
        data['volume_ma{}'.format(window)] = \
            data['volume'].rolling(window).mean()
        data['close_ma%d_ratio' % window] = \
            (data['close'] - data['close_ma%d' % window]) \
            / data['close_ma%d' % window]
        data['volume_ma%d_ratio' % window] = \
            (data['volume'] - data['volume_ma%d' % window]) \
            / data['volume_ma%d' % window]
            
        if ver == 'v1.rich':
            data['inst_ma{}'.format(window)] = \
                data['close'].rolling(window).mean()
            data['frgn_ma{}'.format(window)] = \
                data['volume'].rolling(window).mean()
            data['inst_ma%d_ratio' % window] = \
                (data['close'] - data['inst_ma%d' % window]) \
                / data['inst_ma%d' % window]
            data['frgn_ma%d_ratio' % window] = \
                (data['volume'] - data['frgn_ma%d' % window]) \
                / data['frgn_ma%d' % window]

    data['open_lastclose_ratio'] = np.zeros(len(data))
    data.loc[1:, 'open_lastclose_ratio'] = \
        (data['open'][1:].values - data['close'][:-1].values) \
        / data['close'][:-1].values
    data['high_close_ratio'] = \
        (data['high'].values - data['close'].values) \
        / data['close'].values
    data['low_close_ratio'] = \
        (data['low'].values - data['close'].values) \
        / data['close'].values
    data['close_lastclose_ratio'] = np.zeros(len(data))
    data.loc[1:, 'close_lastclose_ratio'] = \
        (data['close'][1:].values - data['close'][:-1].values) \
        / data['close'][:-1].values
    data['volume_lastvolume_ratio'] = np.zeros(len(data))
    data.loc[1:, 'volume_lastvolume_ratio'] = \
        (data['volume'][1:].values - data['volume'][:-1].values) \
        / data['volume'][:-1] \
            .replace(to_replace=0, method='ffill') \
            .replace(to_replace=0, method='bfill').values

    if ver == 'v1.rich':
        data['inst_lastinst_ratio'] = np.zeros(len(data))
        data.loc[1:, 'inst_lastinst_ratio'] = \
            (data['inst'][1:].values - data['inst'][:-1].values) \
            / data['inst'][:-1] \
                .replace(to_replace=0, method='ffill') \
                .replace(to_replace=0, method='bfill').values
        data['frgn_lastfrgn_ratio'] = np.zeros(len(data))
        data.loc[1:, 'frgn_lastfrgn_ratio'] = \
            (data['frgn'][1:].values - data['frgn'][:-1].values) \
            / data['frgn'][:-1] \
                .replace(to_replace=0, method='ffill') \
                .replace(to_replace=0, method='bfill').values

    return data

In [53]:
def load_data(fpath, date_from, date_to, ver='v2'):
    header = None if ver == 'v1' else 0
    data = pd.read_csv(fpath, thousands=',', header=header, 
        converters={'date': lambda x: str(x)})

    if ver == 'v1':
        data.columns = ['date', 'open', 'high', 'low', 'close', 'volume']

    # 데이터 전처리
    data = preprocess(data)
    
    # 기간 필터링
    data['Date'] = data['Date'].str.replace('-', '')
    data = data[(data['Date'] >= date_from) & (data['Date'] <= date_to)]
    data = data.dropna()

    # 차트 데이터 분리
    chart_data = data[COLUMNS_CHART_DATA]

    # 학습 데이터 분리
    training_data = None
    if ver == 'v1':
        training_data = data[COLUMNS_TRAINING_DATA_V1]
    elif ver == 'v1.rich':
        training_data = data[COLUMNS_TRAINING_DATA_V1_RICH]
    elif ver == 'v2':
        data.loc[:, ['per', 'pbr', 'roe']] = \
            data[['per', 'pbr', 'roe']].apply(lambda x: x / 100)
        training_data = data[COLUMNS_TRAINING_DATA_V2]
        training_data = training_data.apply(np.tanh)
    else:
        raise Exception('Invalid version.')
    
    return chart_data, training_data

In [61]:
d1=load_data(a1,2014-01-01,2019-12-31 ver='v2')

SyntaxError: invalid token (<ipython-input-61-533bf086499c>, line 1)

In [59]:
d

,date,open,high,low,close,volume,close_ma5,volume_ma5,close_ma5_ratio,volume_ma5_ratio,...,volume_ma60_ratio,close_ma120,volume_ma120,close_ma120_ratio,volume_ma120_ratio,open_lastclose_ratio,high_close_ratio,low_close_ratio,close_lastclose_ratio,volume_lastvolume_ratio
0,2014-01-02,277500,281000,276000,279500,101630,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.000000,0.005367,-0.012522,0.000000,0.000000
1,2014-01-03,278000,280000,274500,278500,78910,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,-0.005367,0.005386,-0.014363,-0.003578,-0.223556
2,2014-01-06,277000,278000,272500,277000,42426,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,-0.005386,0.003610,-0.016245,-0.005386,-0.462350
3,2014-01-07,274000,277000,273000,275500,49898,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,-0.010830,0.005445,-0.009074,-0.005415,0.176118
4,2014-01-08,274500,276500,272000,274500,42155,277000.0,63003.8,-0.009025,-0.330913,...,NaN,NaN,NaN,NaN,NaN,-0.003630,0.007286,-0.009107,-0.003630,-0.155177
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1466,2019-12-23,244000,244500,240000,243500,33748,243400.0,55001.0,0.000411,-0.386411,...,-0.213606,247070.833333,41027.991667,-0.014453,-0.177440,0.010352,0.004107,-0.014374,0.008282,-0.628027
1467,2019-12-24,245500,245500,241500,242000,17060,242000.0,53833.2,0.000000,-0.683095,...,-0.602326,246650.000000,40961.900000,-0.018853,-0.583515,0.008214,0.014463,-0.002066,-0.006160,-0.494489
1468,2019-12-26,244000,245000,241500,243000,32651,242200.0,48735.6,0.003303,-0.330038,...,-0.243571,246220.833333,41078.425000,-0.013081,-0.205155,0.008264,0.008230,-0.006173,0.004132,0.913892
1469,2019-12-27,246000,251500,241000,250000,50065,244000.0,44850.2,0.024590,0.116271,...,0.160455,245854.166667,41348.291667,0.016863,0.210812,0.012346,0.006000,-0.036000,0.028807,0.533337


In [50]:
data_manager_py.to_csv('./Stock Data/Data/stock_df/stock_df/hey.csv')

In [47]:
data_manager_py

,date,open,high,low,close,volume,close_ma5,volume_ma5,close_ma5_ratio,volume_ma5_ratio,...,volume_ma60_ratio,close_ma120,volume_ma120,close_ma120_ratio,volume_ma120_ratio,open_lastclose_ratio,high_close_ratio,low_close_ratio,close_lastclose_ratio,volume_lastvolume_ratio
0,2014-01-02,277500,281000,276000,279500,101630,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.000000,0.005367,-0.012522,0.000000,0.000000
1,2014-01-03,278000,280000,274500,278500,78910,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,-0.005367,0.005386,-0.014363,-0.003578,-0.223556
2,2014-01-06,277000,278000,272500,277000,42426,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,-0.005386,0.003610,-0.016245,-0.005386,-0.462350
3,2014-01-07,274000,277000,273000,275500,49898,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,-0.010830,0.005445,-0.009074,-0.005415,0.176118
4,2014-01-08,274500,276500,272000,274500,42155,277000.0,63003.8,-0.009025,-0.330913,...,NaN,NaN,NaN,NaN,NaN,-0.003630,0.007286,-0.009107,-0.003630,-0.155177
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1466,2019-12-23,244000,244500,240000,243500,33748,243400.0,55001.0,0.000411,-0.386411,...,-0.213606,247070.833333,41027.991667,-0.014453,-0.177440,0.010352,0.004107,-0.014374,0.008282,-0.628027
1467,2019-12-24,245500,245500,241500,242000,17060,242000.0,53833.2,0.000000,-0.683095,...,-0.602326,246650.000000,40961.900000,-0.018853,-0.583515,0.008214,0.014463,-0.002066,-0.006160,-0.494489
1468,2019-12-26,244000,245000,241500,243000,32651,242200.0,48735.6,0.003303,-0.330038,...,-0.243571,246220.833333,41078.425000,-0.013081,-0.205155,0.008264,0.008230,-0.006173,0.004132,0.913892
1469,2019-12-27,246000,251500,241000,250000,50065,244000.0,44850.2,0.024590,0.116271,...,0.160455,245854.166667,41348.291667,0.016863,0.210812,0.012346,0.006000,-0.036000,0.028807,0.533337
